In [1]:
#import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder


from scipy.stats.distributions import gamma
from scipy.stats.distributions import poisson
from scipy.stats.distributions import expon

import statistics
from scipy.stats import norm
from scipy import stats
import scipy.stats

import matplotlib.pyplot as plt



In [2]:
##### import data
data = pd.read_csv("../TOKEN BUCKET ALGORITHM/H265-data/BluePlanet/blueplanet-QP 10.csv",infer_datetime_format=True,
                  parse_dates=['arrival_time'], index_col='arrival_time', 
                   names=['decoded_frame','arrival_time', 'video_frame', 
                                         'frame_size', 'x', 'y', 'z'])

In [3]:
data.head()

,decoded_frame,video_frame,frame_size,x,y,z
arrival_time,,,,,,
0.00000,0,I,686.0,99.99,99.99,99.99
0.04167,1,B,29.0,99.99,99.99,99.99
0.08333,2,B,30.0,99.99,99.99,99.99
0.12500,3,B,29.0,99.99,99.99,99.99
0.16667,4,B,30.0,99.99,99.99,99.99


In [4]:
data.dropna(0, inplace=True)

In [5]:
#drop decoded frame column
data.drop(('decoded_frame'), axis=1, inplace=True)

data.drop(('x'), axis=1, inplace=True)
data.drop(('y'), axis=1, inplace=True)
data.drop(('z'), axis=1, inplace=True)

In [6]:
# Label encoding of species column numerically
le = LabelEncoder()
le.fit(data['video_frame'])
print(list(le.classes_))
data['video_frame'] = le.transform(data['video_frame'])

['B', 'I', 'P']


In [7]:
data.head()

,video_frame,frame_size
arrival_time,,
0.00000,1,686.0
0.04167,0,29.0
0.08333,0,30.0
0.12500,0,29.0
0.16667,0,30.0


In [8]:
#Drop NA values
data.frame_size.dropna(0,inplace= True)

In [9]:
data.reset_index(inplace=True)

In [10]:
data.head()

,arrival_time,video_frame,frame_size
0,0.00000,1,686.0
1,0.04167,0,29.0
2,0.08333,0,30.0
3,0.12500,0,29.0
4,0.16667,0,30.0


In [11]:
#Sort the arrival time 

data.sort_values('arrival_time', inplace=True)
data.reset_index(drop=True, inplace=True)

In [12]:
data.head(20)
#I = 1, P=2, B= 0

,arrival_time,video_frame,frame_size
0,0.00000,1,686.0
1,0.04167,0,29.0
2,0.08333,0,30.0
3,0.12500,0,29.0
4,0.16667,0,30.0
5,0.20833,0,29.0
6,0.25000,0,30.0
7,0.29167,0,29.0
8,0.33333,2,43.0
9,0.37500,0,29.0


In [13]:
#Check for missing values

data.isnull().sum()

arrival_time    0
video_frame     0
frame_size      0
dtype: int64

***Define the parameters***

In [14]:
peak_frame_i = max(data[data["video_frame"]==1].frame_size)
peak_frame_b = max(data[data["video_frame"]==0].frame_size)
peak_frame_p = max(data[data["video_frame"]==2].frame_size)

In [15]:
mean_frame_i = np.ceil(np.mean(data[data["video_frame"]== 1].frame_size))
mean_frame_b = np.ceil(np.mean(data[data["video_frame"]== 0].frame_size))
mean_frame_p = np.ceil(np.mean(data[data["video_frame"]== 2].frame_size))

In [16]:
print ('peak_frame i = {}, peak_frame b = {}, peak_frame p = {}'.format( peak_frame_i,peak_frame_b,peak_frame_p))
print ('\nmean_frame i = {}, mean_frame b = {}, mean_frame p = {}'.format(mean_frame_i,mean_frame_b,mean_frame_p))

peak_frame i = 983397.0, peak_frame b = 628340.0, peak_frame p = 842199.0

mean_frame i = 349516.0, mean_frame b = 76061.0, mean_frame p = 262669.0


In [17]:
len(data.frame_size)

61008

In [18]:
peak_frame = max(data.frame_size)
print ('peak_frame', peak_frame)

mean_frame = np.mean(data.frame_size)
mean_frame = np.ceil(mean_frame)
print('mean_frame', mean_frame)

num_of_frames = data.shape[0]
total = np.sum(data.frame_size)
print(num_of_frames)
print(total)

peak_frame 983397.0
mean_frame 103006.0
61008
6284138716.0


***Problem statement***

In [19]:

#Calculate the following token generation rate for each frame type (I, P and B):

#I-frame (token generation of I-frame)
I = peak_frame_i/ mean_frame_i
I= np.ceil(I)
print('I rate', I)

# P-frame (token generation rate of P-frame)
P = I * mean_frame_p
P = np.ceil(P)
print ('P rate', P)

# B-frame (token generation rate of B-frame)
B = I * mean_frame_b
B = np.ceil(B)
print('B rate', B)

I rate 3.0
P rate 788007.0
B rate 228183.0


In [20]:
def token_generation_rate(video_frame):
    if video_frame == 0: #B= 0
        token_generation_rate = mean_frame_b
    elif video_frame == 1: #I =1
        token_generation_rate = mean_frame_i
    else: #P=2
        token_generation_rate = mean_frame_p


In [21]:
#Token Bucket algorithm using mean token generation rate

for m in range(10):  # 
    byte_sent = 0   # incoming packet forwarded 
    deleted_bytes = 0
    bucket_size = 0  #  token bucket_size
    marked_bytes = 0  # non conforming packets marked
    j = 0
    l = 0
    max_bucket_size = m * peak_frame  # maximum bucket size - we increased the bucket size to 2,3,4.. 
                                       
    
    for i in range(data.shape[0]):
        video_frame = data.loc[i,"video_frame"]
        
        if video_frame == 0: #B= 0
            token_generation_rate = B
        elif video_frame == 1: #I =1
            token_generation_rate = I
        else: #P=2
            token_generation_rate = P
        
        #video_frame = data.loc[i,"video_frame"]
        #frame_type = data["video_frame"].iloc[i]
        #token_rate = token_generation_rate(frame_type)
        
        
        bucket_size += token_generation_rate  # Finally assign the above solution HERE (token generation rate)
            
            
        if bucket_size > max_bucket_size: 
            bucket_size = max_bucket_size 
            #l += 1
        
        if data.loc[i, 'frame_size'] <= bucket_size:  
                                                        
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])  
                                                                      
            
        else:
            byte_sent += bucket_size 
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)  
            bucket_size = 0  
            
    percent = marked_bytes * 100 / total
    print(percent)
            


100.0
0.2832024053620525
0.08820249282352093
0.07255361802233011
0.056904743221139295
0.04125586841994848
0.025606993618757668
0.009958118817566853
1.0868633409714822e-05
1.0868633409714822e-05
